<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA6_%ED%83%80%EC%9E%90_%EC%84%B8%EC%9D%B4%EB%B2%84_%EB%A7%A4%ED%8A%B8%EB%A6%AD%EC%8A%A42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/EDA결과/'

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [31]:
team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA3-팀타자_세이버 매트릭스_합본.csv', encoding='utf-8-sig')
hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA3-개인타자_세이버 매트릭스_합본.csv' , encoding='utf-8-sig')

In [32]:
team_hitter = team_hitter[['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수',
       '타점', '득점', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구',
       '고4', '사구', '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌',
       '단타', '출루율', '장타력', 'ops', '타율', 'ISO', 'BABIP', 'BB/K', 'K%', 'BB%']]

In [33]:
team_hitter['일자'] = pd.to_datetime(team_hitter['일자'],format='%Y-%m-%d', errors='raise')
team_hitter['시즌'] = team_hitter['일자'].dt.year 

In [34]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.344885,0.402433,0.747318,0.275152,0.127281,0.310545,0.559367,0.157720,0.084478
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.350258,0.422276,0.772534,0.276029,0.146247,0.310727,0.548469,0.167164,0.089979
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.369847,0.445285,0.815132,0.302491,0.142794,0.333154,0.625683,0.143473,0.086633
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.315240,0.360465,0.675705,0.243256,0.117209,0.283718,0.451193,0.191286,0.082988


wOBA => 가중 출루율, 타자가 얼마나 공격에 기여했는가
*연도별 상수는 스텟티즈에서 참고<br/>
ex>(0.709 x (볼넷 - 고의 볼넷) + 0.749 x 사구 + 0.909 x 1루타 + 1.362 x 2루타 + 1.828 x 3루타 + 2.205 x 홈런) / (타수 + 볼넷 - 고의 볼넷 + 사구 + 희생플라이)

In [35]:
c_dic = {2020: [0.730, 0.759,	0.938, 1.284,	1.595,	1.959],
         2019: [0.721,	0.752,	0.945,	1.317,	1.653,	2.074],
         2018: [0.726,	0.753,	0.920,	1.242,	1.533,	1.857],
         2017: [0.728,	0.756,	0.927,	1.259,	1.558,	1.901],
         2016: [0.747,	0.774,	0.944,	1.273,	1.569,	1.898]}


In [36]:
def createwOBA(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wOBA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic[row['시즌']]
    up = (ct[0] * (previous_sum['4구'] - previous_sum['고4']) + ct[1] * previous_sum['사구'] + ct[2] * previous_sum['단타'] + ct[3] * previous_sum['2루타'] + ct[4] * previous_sum['3루타'] + ct[5] * previous_sum['홈런'] )
    down = (previous_sum['타수'] + previous_sum['4구'] - previous_sum['고4'] + previous_sum['사구'] + previous_sum['희비'])
    if down == 0:
        row['wOBA'] = 0
        return row
    row['wOBA'] = up / down
    return row

In [38]:
team_hitter = team_hitter.apply(createwOBA, axis=1)

wRAA => 리그평균 대비 득점생산, 리그 평균과 비교해서 한 타자가 팀에 기여하는 득점수 

In [44]:
c_dic2 = {2020: [0.345,	1.153, 4328, 56881],
         2019: [0.335,	1.242, 6548, 55612],
         2018: [0.351,	1.075, 7994, 56903],
         2017: [0.351,	1.107, 7682, 56881],
         2016: [0.362,	1.097, 8074, 57614]}
def createwRAA(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRAA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = row['wOBA'] - ct[0]
    down = ct[1]
    row['wRAA'] = up / down * previous_sum['타자']
    return row

In [41]:
team_hitter = team_hitter.apply(createwRAA, axis=1)

In [43]:
team_hitter.iloc[2870: 2890]

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%,wOBA,wRAA
2870,20171003HTKT0,2017-10-03,KT,HT,0,B,39,37,2,2,12,1,0,1,0,0,0,1,1,0,0,9,1,0,10,0.000000,7,0,2017,10,0.331971,0.410000,0.741971,0.275102,0.134898,0.322158,0.370040,0.185465,0.066605,0.331222,-97.301349
2871,20171003HTKT0,2017-10-03,HT,KT,0,T,46,37,10,10,13,3,0,2,2,0,1,1,7,0,0,5,0,0,9,0.384615,13,5,2017,8,0.369622,0.468560,0.838182,0.301861,0.166699,0.334389,0.555305,0.153393,0.081891,0.370153,100.262105
2872,20171003LGLT0,2017-10-03,LT,LG,0,B,32,28,3,4,6,3,0,0,0,0,2,0,2,0,0,4,0,0,4,0.200000,10,2,2017,3,0.356873,0.433548,0.790421,0.285743,0.147805,0.330811,0.448718,0.180878,0.075276,0.351971,4.948459
2873,20171003LGLT0,2017-10-03,LG,LT,0,T,36,33,2,2,6,3,1,1,0,0,0,0,3,0,0,9,0,1,7,0.100000,10,1,2017,1,0.348964,0.399511,0.748475,0.281816,0.117695,0.331599,0.429842,0.181982,0.075166,0.337739,-66.819569
2874,20171003NCHH0,2017-10-03,HH,NC,0,B,55,52,8,8,15,2,0,2,0,0,0,0,3,1,0,6,0,2,11,0.285714,14,4,2017,11,0.350715,0.434914,0.785629,0.287264,0.147650,0.328718,0.426653,0.172981,0.071301,0.349861,-5.774237
2875,20171003NCHH0,2017-10-03,NC,HH,0,T,54,48,8,8,16,1,0,1,1,1,0,0,4,0,2,8,1,1,10,0.555556,9,5,2017,14,0.364470,0.443451,0.807920,0.292785,0.150666,0.334596,0.480947,0.169844,0.078363,0.360063,46.960343
2876,20171003SKOB0,2017-10-03,OB,SK,0,B,36,31,2,2,6,0,0,1,0,0,1,0,3,0,1,5,1,2,7,0.166667,6,1,2017,5,0.369565,0.460264,0.829829,0.294419,0.165845,0.330156,0.563351,0.165282,0.089823,0.367748,87.704952
2877,20171003SKOB0,2017-10-03,SK,OB,0,T,38,35,3,3,8,1,0,0,1,0,0,0,3,0,0,8,0,0,8,0.200000,10,2,2017,7,0.341258,0.466667,0.807925,0.271779,0.194888,0.303745,0.388278,0.198329,0.073375,0.352841,9.188413
2878,20171003WOSS0,2017-10-03,SS,WO,0,B,41,37,10,10,12,3,1,2,0,0,0,0,4,0,0,5,0,1,7,0.461538,13,6,2017,6,0.339158,0.426651,0.765809,0.278173,0.148478,0.316633,0.434560,0.173128,0.072225,0.339700,-57.836028
2879,20171003WOSS0,2017-10-03,WO,SS,0,T,45,41,8,9,15,3,0,3,0,0,0,0,4,0,0,12,1,2,9,0.307692,13,4,2017,9,0.356181,0.435436,0.791617,0.289500,0.145936,0.339391,0.440228,0.186483,0.079441,0.352999,10.233937


In [48]:
def createwRC(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRC'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = ct[2]
    down = ct[3]
    row['wRC'] = (up / down) * previous_sum['타자'] + row['wRAA']
    return row

In [49]:
team_hitter = team_hitter.apply(createwRC, axis=1)

In [51]:
team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA6-팀타자_세이버 매트릭스2_합본.csv', encoding='utf-8-sig')

GPA = > 출루율에 1.8의 가중치를 부여한 OPS 변형

In [52]:
team_hitter['GPA'] = (1.8 * team_hitter['출루율'] + team_hitter['장타력']) / 4 

In [58]:
team_hitter.iloc[2870: 2890]

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%,wOBA,wRAA,wRC,GPA,HR%
2870,20171003HTKT0,2017-10-03,KT,HT,0,B,39,37,2,2,12,1,0,1,0,0,0,1,1,0,0,9,1,0,10,0.000000,7,0,2017,10,0.331971,0.410000,0.741971,0.275102,0.134898,0.322158,0.370040,0.185465,0.066605,0.331222,-97.301349,638.202105,0.251887,0.021711
2871,20171003HTKT0,2017-10-03,HT,KT,0,T,46,37,10,10,13,3,0,2,2,0,1,1,7,0,0,5,0,0,9,0.384615,13,5,2017,8,0.369622,0.468560,0.838182,0.301861,0.166699,0.334389,0.555305,0.153393,0.081891,0.370153,100.262105,882.899365,0.283470,0.029086
2872,20171003LGLT0,2017-10-03,LT,LG,0,B,32,28,3,4,6,3,0,0,0,0,2,0,2,0,0,4,0,0,4,0.200000,10,2,2017,3,0.356873,0.433548,0.790421,0.285743,0.147805,0.330811,0.448718,0.180878,0.075276,0.351971,4.948459,766.517313,0.268980,0.026935
2873,20171003LGLT0,2017-10-03,LG,LT,0,T,36,33,2,2,6,3,1,1,0,0,0,0,3,0,0,9,0,1,7,0.100000,10,1,2017,1,0.348964,0.399511,0.748475,0.281816,0.117695,0.331599,0.429842,0.181982,0.075166,0.337739,-66.819569,686.510998,0.256912,0.019601
2874,20171003NCHH0,2017-10-03,HH,NC,0,B,55,52,8,8,15,2,0,2,0,0,0,0,3,1,0,6,0,2,11,0.285714,14,4,2017,11,0.350715,0.434914,0.785629,0.287264,0.147650,0.328718,0.426653,0.172981,0.071301,0.349861,-5.774237,751.878055,0.266550,0.026447
2875,20171003NCHH0,2017-10-03,NC,HH,0,T,54,48,8,8,16,1,0,1,1,1,0,0,4,0,2,8,1,1,10,0.555556,9,5,2017,14,0.364470,0.443451,0.807920,0.292785,0.150666,0.334596,0.480947,0.169844,0.078363,0.360063,46.960343,821.629424,0.274874,0.025888
2876,20171003SKOB0,2017-10-03,OB,SK,0,B,36,31,2,2,6,0,0,1,0,0,1,0,3,0,1,5,1,2,7,0.166667,6,1,2017,5,0.369565,0.460264,0.829829,0.294419,0.165845,0.330156,0.563351,0.165282,0.089823,0.367748,87.704952,870.612320,0.281370,0.030633
2877,20171003SKOB0,2017-10-03,SK,OB,0,T,38,35,3,3,8,1,0,0,1,0,0,0,3,0,0,8,0,0,8,0.200000,10,2,2017,7,0.341258,0.466667,0.807925,0.271779,0.194888,0.303745,0.388278,0.198329,0.073375,0.352841,9.188413,755.496178,0.270233,0.042499
2878,20171003WOSS0,2017-10-03,SS,WO,0,B,41,37,10,10,12,3,1,2,0,0,0,0,4,0,0,5,0,1,7,0.461538,13,6,2017,6,0.339158,0.426651,0.765809,0.278173,0.148478,0.316633,0.434560,0.173128,0.072225,0.339700,-57.836028,707.379282,0.259284,0.025314
2879,20171003WOSS0,2017-10-03,WO,SS,0,T,45,41,8,9,15,3,0,3,0,0,0,0,4,0,0,12,1,2,9,0.307692,13,4,2017,9,0.356181,0.435436,0.791617,0.289500,0.145936,0.339391,0.440228,0.186483,0.079441,0.352999,10.233937,775.584300,0.269141,0.024416


HR% => 타석당 홈런비율

In [55]:
def createHR(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['HR%'] = 0
        return row
    previous_sum = previous.sum()
    up_hr = previous_sum['홈런']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['HR%'] = 0
        return row
    row['HR%'] = up_hr / down
    return row

In [56]:
team_hitter = team_hitter.apply(createHR, axis=1)

In [57]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%,wOBA,wRAA,wRC,GPA,HR%
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.344885,0.402433,0.747318,0.275152,0.127281,0.310545,0.559367,0.157720,0.084478,0.340581,-9.244490,174.281397,0.255807,0.021640
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.350258,0.422276,0.772534,0.276029,0.146247,0.310727,0.548469,0.167164,0.089979,0.350847,11.912901,190.645202,0.263185,0.028145
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.369847,0.445285,0.815132,0.302491,0.142794,0.333154,0.625683,0.143473,0.086633,0.367820,50.648229,245.359152,0.277752,0.023912
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.315240,0.360465,0.675705,0.243256,0.117209,0.283718,0.451193,0.191286,0.082988,0.310297,-72.776044,111.206375,0.231974,0.023237


In [60]:
team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA6-팀타자_세이버 매트릭스2_합본.csv', encoding='utf-8-sig')

In [61]:
hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA3-개인타자_세이버 매트릭스_합본.csv' , encoding='utf-8-sig')

In [62]:
hitter['일자'] = pd.to_datetime(hitter['일자'],format='%Y-%m-%d', errors='raise')
hitter['시즌'] = hitter['일자'].dt.year 

In [ ]:
c_dic = {2020: [0.730, 0.759,	0.938, 1.284,	1.595,	1.959],
         2019: [0.721,	0.752,	0.945,	1.317,	1.653,	2.074],
         2018: [0.726,	0.753,	0.920,	1.242,	1.533,	1.857],
         2017: [0.728,	0.756,	0.927,	1.259,	1.558,	1.901],
         2016: [0.747,	0.774,	0.944,	1.273,	1.569,	1.898]}

c_dic2 = {2020: [0.345,	1.153, 4328, 56881],
         2019: [0.335,	1.242, 6548, 55612],
         2018: [0.351,	1.075, 7994, 56903],
         2017: [0.351,	1.107, 7682, 56881],
         2016: [0.362,	1.097, 8074, 57614]}

In [63]:
def createwOBA(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wOBA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic[row['시즌']]
    up = (ct[0] * (previous_sum['4구'] - previous_sum['고4']) + ct[1] * previous_sum['사구'] + ct[2] * previous_sum['단타'] + ct[3] * previous_sum['2루타'] + ct[4] * previous_sum['3루타'] + ct[5] * previous_sum['홈런'] )
    down = (previous_sum['타수'] + previous_sum['4구'] - previous_sum['고4'] + previous_sum['사구'] + previous_sum['희비'])
    if down == 0:
        row['wOBA'] = 0
        return row
    row['wOBA'] = up / down
    return row

def createwRAA(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRAA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = row['wOBA'] - ct[0]
    down = ct[1]
    row['wRAA'] = up / down * previous_sum['타자']
    return row

def createwRC(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['wRC'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = ct[2]
    down = ct[3]
    row['wRC'] = (up / down) * previous_sum['타자'] + row['wRAA']
    return row

def createHR(row):
    previous = hitter[(hitter['시즌'] == row['시즌']) & (hitter['선수코드'] == row['선수코드']) & (hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['HR%'] = 0
        return row
    previous_sum = previous.sum()
    up_hr = previous_sum['홈런']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['HR%'] = 0
        return row
        
    row['HR%'] = up_hr / down
    return row

In [64]:
hitter = hitter.apply(createwOBA, axis=1)
hitter = hitter.apply(createwRAA, axis=1)
hitter = hitter.apply(createwRC, axis=1)
hitter = hitter.apply(createHR, axis=1)
hitter['GPA'] = (1.8 * hitter['출루율'] + hitter['장타력']) / 4 

In [65]:
hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA6-개인타자_세이버 매트릭스2_합본.csv', encoding='utf-8-sig')

In [66]:
hitter

,Unnamed: 0,Unnamed: 0.1,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,선수코드,선발,타순,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,BABIP,BB/K,ISO,K%,BB%,wOBA,wRAA,wRC,HR%,GPA
0,0,0,20160401HHLG0,2016-04-01,HH,LG,0,T,60404,0,3,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0,0,2016,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,20160401HHLG0,2016-04-01,LG,HH,0,B,61102,1,8,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.0,0,0,2016,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,2,20160401HHLG0,2016-04-01,LG,HH,0,B,61186,1,7,4,4,2,2,3,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1.0,1,1,2016,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,3,20160401HHLG0,2016-04-01,LG,HH,0,B,62164,0,9,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,2016,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,4,20160401HHLG0,2016-04-01,HH,LG,0,T,62700,1,9,2,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0,0,2016,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81097,81097,8055,20200719WOSK0,2020-07-19,SK,WO,0,B,75847,1,3,4,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0.0,1,0,2020,1,0.416327,0.545000,0.961327,0.285000,0.293333,0.972973,0.260000,0.151639,0.143443,0.424664,16.927722,35.569448,0.053279,0.323597
81098,81098,8056,20200719WOSK0,2020-07-19,SK,WO,0,B,76802,1,9,4,3,1,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,4,1.0,1,1,2020,3,0.294964,0.225806,0.520770,0.217742,0.236842,1.090909,0.008065,0.075862,0.082759,0.258633,-10.861406,0.171452,0.000000,0.189185
81099,81099,8057,20200719WOSK0,2020-07-19,SK,WO,0,B,77463,0,6,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0,0,2020,0,0.350000,0.410714,0.760714,0.303571,0.340909,0.400000,0.107143,0.169492,0.050847,0.342000,-0.156114,4.409206,0.033898,0.260179
81100,81100,8058,20200719WOSK0,2020-07-19,WO,SK,0,T,78168,1,1,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,1,0,2020,0,0.402930,0.449782,0.852712,0.301310,0.321608,1.357143,0.148472,0.102564,0.139194,0.389168,10.457936,31.230144,0.018315,0.293764
